In [1]:
from Define_Model.TDNN.TDNN import TDNN_v5
import kaldi_io
import torch

from Define_Model.Loss.SoftmaxLoss import AdditiveMarginLinear
import numpy as np
import kaldi_io
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
plt.rc('font', family='Times New Roman')

In [2]:
model = TDNN_v5(num_classes=1211, embedding_size=512, input_dim=40, alpha=0., input_norm='Mean',
                 encoder_type='STAP', activation='relu',
                 num_classes_b=0, block_type='basic', first_2d=False, stride=[1],
                 mask='None', mask_len=20, channels=[512, 512, 512, 512, 1500])

# model.classifier = AdditiveMarginLinear(feat_dim=512, num_classes=1211)
ckp = torch.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/checkpoint/TDNN_v5/vox1/klfb_egs_baseline/soft/featcombined_inputMean_STAP_em512_wde3_var/checkpoint_50.pth')
model.load_state_dict(ckp['state_dict'][0])

model.eval()

/home/yangwenhao/anaconda3/envs/py37/lib/python3.7/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.loss.CrossEntropyLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


TDNN_v5(
  (inst_layer): Mean_Norm(dim=-2)
  (frame1): TimeDelayLayer_v5(
    (kernel): Conv1d(40, 512, kernel_size=(5,), stride=(1,))
    (nonlinearity): ReLU(inplace=True)
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (frame2): TimeDelayLayer_v5(
    (kernel): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(2,))
    (nonlinearity): ReLU(inplace=True)
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (frame3): TimeDelayLayer_v5(
    (kernel): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(3,))
    (nonlinearity): ReLU(inplace=True)
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (frame4): TimeDelayLayer_v5(
    (kernel): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
    (nonlinearity): ReLU(inplace=True)
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (frame5): TimeDela

In [3]:
stmx = torch.nn.Softmax(dim=1)

In [11]:
x0 = kaldi_io.read_mat('/home/yangwenhao/local/project/lstm_speaker_verification/data/vox1/klfb/fbank/dev_fb40/raw_fbank_dev_fb40.1.ark:69000')
x01 = kaldi_io.read_mat('/home/yangwenhao/local/project/lstm_speaker_verification/data/vox1/klfb/fbank/dev_fb40/raw_fbank_dev_fb40.1.ark:2686603')
x02 = kaldi_io.read_mat('/home/yangwenhao/local/project/lstm_speaker_verification/data/vox1/klfb/fbank/dev_fb40/raw_fbank_dev_fb40.1.ark:6070422')


x1 = kaldi_io.read_mat('/home/yangwenhao/local/project/lstm_speaker_verification/data/vox1/dev_aug_spk/fbank/raw_fbank_dev_aug_spk.1.ark:20')
x2 = kaldi_io.read_mat('/home/yangwenhao/local/project/lstm_speaker_verification/data/vox1/dev_aug_spk/fbank/raw_fbank_dev_aug_spk.1.ark:23267')


x0 = torch.FloatTensor(x0)[:,:40].unsqueeze(0)
print(x0.shape)
x01 = torch.FloatTensor(x01)[:,:40].unsqueeze(0)
x02 = torch.FloatTensor(x02)[:,:40].unsqueeze(0)
print(x01.shape)
print(x02.shape)



x1 = torch.FloatTensor(x1)[:,:40].unsqueeze(0)
x2 = torch.FloatTensor(x2)[:,:40].unsqueeze(0)
print(x1.shape)
print(x2.shape)

torch.Size([1, 558, 40])
torch.Size([1, 414, 40])
torch.Size([1, 434, 40])
torch.Size([1, 558, 40])
torch.Size([1, 558, 40])


In [5]:
logits0, embedding_b0 = model(x0)
logits01, embedding_b01 = model(x01)
logits02, embedding_b02 = model(x02)

logits1, embedding_b1 = model(x1)
logits2, embedding_b2 = model(x2)

In [6]:
stmx(logits0)[0].sort()

torch.return_types.sort(
values=tensor([9.3096e-08, 1.3845e-07, 2.0548e-07,  ..., 1.4585e-01, 1.4924e-01,
        1.6458e-01], grad_fn=<SortBackward>),
indices=tensor([ 674,  676, 1139,  ...,  654,    0,  518]))

In [7]:
stmx(logits01)[0].sort()

torch.return_types.sort(
values=tensor([2.6092e-07, 3.1600e-07, 4.3541e-07,  ..., 4.0313e-02, 6.7129e-02,
        3.4266e-01], grad_fn=<SortBackward>),
indices=tensor([1140,  294,  657,  ...,  313,  994,    1]))

In [8]:
stmx(logits02)[0].sort()

torch.return_types.sort(
values=tensor([1.8703e-06, 1.8918e-06, 1.9672e-06,  ..., 4.2330e-02, 4.4083e-02,
        2.0894e-01], grad_fn=<SortBackward>),
indices=tensor([ 398, 1063,  377,  ..., 1074,  260,  928]))

In [9]:
stmx(logits1)[0].sort()

torch.return_types.sort(
values=tensor([7.3378e-08, 1.1264e-07, 1.2866e-07,  ..., 1.0503e-01, 1.1848e-01,
        2.5894e-01], grad_fn=<SortBackward>),
indices=tensor([ 676,  674, 1139,  ...,  405,  654,  618]))

In [10]:
stmx(logits2)[0].sort()

torch.return_types.sort(
values=tensor([8.8413e-07, 1.2508e-06, 1.4454e-06,  ..., 6.4731e-02, 7.1566e-02,
        1.1731e-01], grad_fn=<SortBackward>),
indices=tensor([ 41,  91, 110,  ..., 654, 673,   0]))